In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
df=pd.read_csv("C:\\Users\\LENOVO\\Downloads\\play_tennis.csv")
df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [8]:
def Entropy(data):
  p,n = 0,0

  for i in data.index:
    if(data.loc[i]["play"] ==  "Yes"):p+=1
    elif(data.loc[i]["play"] == "No"):n+=1

  total = p+n

  if p == 0 or n == 0 : return 0

  p,n = p/total,n/total

  return -p*math.log(p,2)-n*math.log(n,2)

def EntropyAttr(data,attr):
  ent = []

  if attr not in data.columns :
    print("Invalid Attribute")
    return

  values = list(set(data[attr]))
  # print(values)
  n = len(data)

  for value in values:
    # print(value)
    pos= 0
    neg= 0

    for i in data.index:
      # print(data[attr][i],data["play"][i] ==  "Yes",value)
      if(data.loc[i][attr] == value and data.loc[i]["play"] ==  "Yes" ):pos+=1
      elif(data.loc[i][attr] == value and data.loc[i]["play"] == "No"):neg+=1
      # else: print(f"Invalid attribute value in {attr} column")
    totV = pos+neg
    if(pos == 0 or neg == 0) : continue
    # print(attr,value,totV,pos,neg)
    pos,neg = pos/totV,neg/totV
    # print((-pos*math.log(pos,2) - neg*math.log(neg,2)))

    ent.append((totV/n)*(-pos*math.log(pos,2) - neg*math.log(neg,2)))
  return sum(ent)

def InfoGain(data,attr):
  return Entropy(data) - EntropyAttr(data,attr)

In [7]:
class Node:
    def __init__(self):
        self.children=[]
        self.value=""
        self.isLeaf=False
        self.pred=""

In [9]:
def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

def classify(root: Node, new):
    for child in root.children:
        if child.value == new[root.value]:
            if child.isLeaf:
                print ("Predicted Label for new example", new," is:", child.pred)
                exit
            else:
                classify (child.children[0], new)

In [10]:
def ID3(examples, attrs):
    root = Node()

    max_gain = 0
    max_feat = ""
    for feature in attrs:
        #print ("\n",examples)
        gain = InfoGain(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value = max_feat
    #print ("\nMax feature attr",max_feat)
    uniq = np.unique(examples[max_feat])
    #print ("\n",uniq)
    for u in uniq:
        #print ("\n",u)
        subdata = examples[examples[max_feat] == u]
        #print ("\n",subdata)
        if Entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["play"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            subdata.reset_index().drop('index',axis=1)
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)

    return root

In [12]:
attrs = data.columns.tolist()
attrs.remove('play')
print(attrs)

['outlook', 'temp', 'humidity', 'wind']


In [14]:
printTree(ID3(data,attrs))

outlook
	Overcast ->  ['Yes']

	Rain
		wind
			Strong ->  ['No']

			Weak ->  ['Yes']

	Sunny
		humidity
			High ->  ['No']

			Normal ->  ['Yes']

